In [1]:
import pandas as pd

In [2]:
pnyc=pd.read_csv('nycpta_cleaned.csv', index_col= 'ein')

In [3]:
snyc=pd.read_csv('nycschool_cleaned.csv', index_col= 'dbn')

/Users/xiangyuren/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,126,127,128,150,151,152,185,223,224,228,229,232,233,234,236,237,249,303,304,305,306,307) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
search_values = ['High school', 'Secondary School', 'Junior High-Intermediate-Middle']

In [5]:
snyc1 = snyc[snyc.location_category_description.str.contains('|'.join(search_values), na=False)]

In [6]:
#snyc1.to_csv("nycschool_clean.csv", encoding='utf8')

In [7]:
#snyc2 = pd.read_csv('nycschool_clean.csv', index_col= 'dbn')

In [8]:


#snyc = snyc.drop(snyc[snyc.location_category_description == 'District Pre-K Center', 'Early Childhood', 'Elementary', 'Junior High-Intermediate-Middle', 'K-12 all grades', 'K-8'].index)

In [9]:
pnyc.head(2)

,x,x_x,name,state,zip5,fips,taxper,fisyr,naics,ntee_final,...,totrev,exps,netinc,ass_boy,ass_eoy,fundbal,x_y,url,address,yearm
ein,,,,,,,,,,,,,,,,,,,,,
43644362,1,1,PTA NEW YORK CONGRESS,NY,10312,36085,200906.0,2009,813410,B94,...,51385.0,69884,-18499.0,27109.0,8610,8610,1,https://www.guidestar.org/profile/04-3644362,255 ionia ave,2009
43644362,2,2,PTA NEW YORK CONGRESS,NY,10312,36085,201006.0,2010,813410,B94,...,77442.0,62591,14851.0,8718.0,23569,23569,1,https://www.guidestar.org/profile/04-3644362,255 ionia ave,2010


In [10]:
snyc1.head(2)

,school_admission,year,schnam,year_2,schlev,schlev5,schs219962016,xdbn,xdbn_flag,enrnumprk,...,bco_executive_director,bco_executive_director_title,district_2,code,dbn_1,school,campus,comments,address,yearm
dbn,,,,,,,,,,,,,,,,,,,,,
01M450,NaN,NaN,EAST SIDE COMMUNITY HIGH SCHOO,1996,NaN,NaN,NaN,01M450,0,0,...,"CHU, YUET",FSC Executive Director,NaN,NaN,NaN,NaN,NaN,NaN,420 e 12 st,1996
01M515,NaN,NaN,LOWER EAST SIDE PREPARATORY HI,1996,NaN,NaN,NaN,01M515,0,0,...,"CHU, YUET",FSC Executive Director,NaN,NaN,NaN,NaN,NaN,NaN,145 stanton st,1996


In [11]:
pnyc.columns

Index(['x', 'x_x', 'name', 'state', 'zip5', 'fips', 'taxper', 'fisyr', 'naics',
       'ntee_final', 'grrec', 'cont', 'progrev', 'invinc', 'fundinc', 'grprof',
       'othinc', 'totrev', 'exps', 'netinc', 'ass_boy', 'ass_eoy', 'fundbal',
       'x_y', 'url', 'address', 'yearm'],
      dtype='object')

In [12]:
snyc1.columns

Index(['school_admission', 'year', 'schnam', 'year_2', 'schlev', 'schlev5',
       'schs219962016', 'xdbn', 'xdbn_flag', 'enrnumprk',
       ...
       'bco_executive_director', 'bco_executive_director_title', 'district_2',
       'code', 'dbn_1', 'school', 'campus', 'comments', 'address', 'yearm'],
      dtype='object', length=309)

In [13]:
snyc1 = snyc.rename(columns={'year':'year1', 'year_2': 'year'})

In [14]:
#pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [15]:
import recordlinkage

In [16]:
indexer = recordlinkage.Index()
indexer.full()

<Index>

In [17]:
snyc1.index.has_duplicates
pnyc.index.has_duplicates

True

In [18]:

snyc1 = snyc1.reset_index()
pnyc = pnyc.reset_index()

In [19]:
candidates = indexer.index(pnyc, snyc1)
print(len(candidates))

705891404


In [20]:
left_on = [ "address", "yearm", "zip5"]

right_on = [
     "address", "year",
    "zip"
]


In [21]:
indexer = recordlinkage.Index()
indexer.block(left_on='zip5', right_on='zip')
candidates = indexer.index(pnyc, snyc1)
print(len(candidates))

3044382


In [22]:
compare = recordlinkage.Compare()
compare.exact('yearm', 'year', label ='Year')
compare.string('address',
            'address',
            method='jarowinkler',
            threshold=0.85,
            label ='Address')
features = compare.compute(candidates,pnyc,snyc1)

In [23]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0       5255
1.0     238973
0.0    2800154
dtype: int64

In [24]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'Year':'Address'].sum(axis=1)
print(potential_matches)

      level_0  level_1  Year  Address  Score
0           0    28219     1      1.0    2.0
1           1    30343     1      1.0    2.0
2           2    32491     1      1.0    2.0
3         343    30370     1      1.0    2.0
4         427    11462     1      1.0    2.0
...       ...      ...   ...      ...    ...
5250    17448    33546     1      1.0    2.0
5251    17449    35685     1      1.0    2.0
5252    17449    35693     1      1.0    2.0
5253    17010    28143     1      1.0    2.0
5254    17011    32414     1      1.0    2.0

[5255 rows x 5 columns]


In [25]:
potential_matches.head(50)
potential_matches.columns

Index(['level_0', 'level_1', 'Year', 'Address', 'Score'], dtype='object')

In [26]:
pnyc.loc[8590,:]

ein                                              133191412
x                                                     9981
x_x                                                   9981
name                          ROCKING HORSE NURSERY SCHOOL
state                                                   NY
zip5                                                 10023
fips                                                 36061
taxper                                            201006.0
fisyr                                                 2010
naics                                               611110
ntee_final                                             B21
grrec                                               174088
cont                                                     0
progrev                                           174088.0
invinc                                                   0
fundinc                                                0.0
grprof                                                  

In [27]:
snyc1.loc[28694,:]

dbn                               03M859
school_admission                     NaN
year1                             2010.0
schnam              SPECIAL MUSIC SCHOOL
year                                2010
                            ...         
school              SPECIAL MUSIC SCHOOL
campus                               NaN
comments               2013 New Programs
address                      129 w 67 st
yearm                               2010
Name: 28694, Length: 310, dtype: object

In [28]:
pnyc['address'].fillna('', inplace=True)
snyc1['address'].fillna('', inplace=True)
pnyc['zip5'].fillna('', inplace=True)
snyc1['zip'].fillna('', inplace=True)
pnyc['yearm'].fillna('', inplace=True)
snyc1['year'].fillna('', inplace=True)

In [29]:
pnyc['yearm']=pnyc['yearm'].map(str)
pnyc['address']=pnyc['address'].map(str)
pnyc['zip5']=pnyc['zip5'].map(str)
pnyc['ein']=pnyc['ein'].map(str)
pnyc['name']=pnyc['name'].map(str)

In [42]:
snyc1['year']=snyc1['year'].map(str)
snyc1['address']=snyc1['address'].map(str)
snyc1['zip']=snyc1['zip'].map(str)
snyc1['dbn']=snyc1['dbn'].map(str)
snyc1['schnam']=snyc1['schnam'].map(str)
snyc1['location_category_description'] = snyc1['location_category_description'].map(str)

In [43]:
pnyc['pta_address_Lookup'] = pnyc[[
    'address'
]].apply(lambda x: '_'.join(x), axis=1)

pnyc['pta_year_Lookup'] = pnyc[[
    'yearm'
]].apply(lambda x: '_'.join(x), axis=1)

pnyc['pta_ein_Lookup'] = pnyc[[
    'ein'
]].apply(lambda x: '_'.join(x), axis=1)

pnyc['pta_zip_Lookup'] = pnyc[[
    'zip5'
]].apply(lambda x: '_'.join(x), axis=1)

pnyc['pta_name_Lookup'] = pnyc[[
    'name'
]].apply(lambda x: '_'.join(x), axis=1)

snyc1['school_address_Lookup'] = snyc1[[
    'address'
]].apply(lambda x: '_'.join(x), axis=1)

snyc1['school_year_Lookup'] = snyc1[[
    'year'
]].apply(lambda x: '_'.join(x), axis=1)

snyc1['school_dbn_Lookup'] = snyc1[[
    'dbn'
]].apply(lambda x: '_'.join(x), axis=1)

snyc1['school_zip_Lookup'] = snyc1[[
    'zip'
]].apply(lambda x: '_'.join(x), axis=1)

snyc1['school_name_Lookup'] = snyc1[[
    'schnam'
]].apply(lambda x: '_'.join(x), axis=1)

snyc1['school_cate_Lookup'] = snyc1[[
    'location_category_description'
]].apply(lambda x: '_'.join(x), axis=1)

pta_lookup = pnyc[['pta_address_Lookup']].reset_index()
pta_year_lookup = pnyc[['pta_year_Lookup']].reset_index()
pta_ein_lookup = pnyc[['pta_ein_Lookup']].reset_index()
pta_zip_lookup = pnyc[['pta_zip_Lookup']].reset_index()
pta_name_lookup = pnyc[['pta_name_Lookup']].reset_index()

school_lookup = snyc1[['school_address_Lookup']].reset_index()
school_year_lookup = snyc1[['school_year_Lookup']].reset_index()
school_dbn_lookup = snyc1[['school_dbn_Lookup']].reset_index()
school_zip_lookup = snyc1[['school_zip_Lookup']].reset_index()
school_cate_lookup = snyc1[['school_cate_Lookup']].reset_index()

In [44]:
school_name_lookup = snyc1[['school_name_Lookup']].reset_index()

In [45]:
pta_lookup.head()
pta_lookup = pta_lookup.rename(columns={'index':'level_0'})

In [46]:
pta_year_lookup.head()
pta_year_lookup = pta_year_lookup.rename(columns={'index':'level_0'})
pta_ein_lookup = pta_ein_lookup.rename(columns={'index':'level_0'})
pta_zip_lookup = pta_zip_lookup.rename(columns={'index':'level_0'})
pta_name_lookup = pta_name_lookup.rename(columns={'index':'level_0'})

In [47]:
school_lookup.head()
school_lookup = school_lookup.rename(columns={'index':'level_1'})
school_year_lookup = school_year_lookup.rename(columns={'index':'level_1'})
school_dbn_lookup = school_dbn_lookup.rename(columns={'index':'level_1'})
school_zip_lookup = school_zip_lookup.rename(columns={'index':'level_1'})
school_name_lookup = school_name_lookup.rename(columns={'index':'level_1'})
school_cate_lookup = school_cate_lookup.rename(columns={'index':'level_1'})

In [48]:
school_merge = potential_matches.merge(pta_lookup, on='level_0',how='left')
school_merge.head()

,level_0,level_1,Year,Address,Score,pta_address_Lookup
0,0,28219,1,1.0,2.0,255 ionia ave
1,1,30343,1,1.0,2.0,255 ionia ave
2,2,32491,1,1.0,2.0,255 ionia ave
3,343,30370,1,1.0,2.0,455 huguenot ave
4,427,11462,1,1.0,2.0,380 genesee ave


In [49]:
school_merge = school_merge.merge(pta_year_lookup, on='level_0',how='left')
school_merge = school_merge.merge(pta_ein_lookup, on='level_0',how='left')
school_merge = school_merge.merge(pta_zip_lookup, on='level_0',how='left')
school_merge = school_merge.merge(pta_name_lookup, on='level_0',how='left')

In [50]:
final_merge = school_merge.merge(school_lookup, on='level_1', how='left')
final_merge = final_merge.merge(school_year_lookup, on='level_1', how='left')
final_merge = final_merge.merge(school_dbn_lookup, on='level_1', how='left')
final_merge = final_merge.merge(school_zip_lookup, on='level_1', how='left')
final_merge = final_merge.merge(school_name_lookup, on='level_1', how='left')
final_merge = final_merge.merge(school_cate_lookup, on='level_1', how='left')

In [51]:
cols = ['level_0', 'level_1', 'Score',
        'pta_address_Lookup', 'school_address_Lookup', 'pta_year_Lookup','pta_zip_Lookup','pta_ein_Lookup', 'school_year_Lookup', 'school_dbn_Lookup', 'school_zip_Lookup', 'school_name_Lookup', 'pta_name_Lookup', 'school_cate_Lookup']
final_merge[cols].sort_values(by=['level_0', 'Score'], ascending=False)

,level_0,level_1,Score,pta_address_Lookup,school_address_Lookup,pta_year_Lookup,pta_zip_Lookup,pta_ein_Lookup,school_year_Lookup,school_dbn_Lookup,school_zip_Lookup,school_name_Lookup,pta_name_Lookup,school_cate_Lookup
3330,19588,7371,2.0,120 w 83 st,100 w 84 st,2002,10024,912170392,2002,03M009,10024.0,SARAH ANDERSON,ENGINE COMPANY 74 FAMILY FUND INC,Elementary
3331,19588,7377,2.0,120 w 83 st,160 w 78 st,2002,10024,912170392,2002,03M087,10024.0,WILLIAM SHERMAN,ENGINE COMPANY 74 FAMILY FUND INC,Elementary
3332,19588,7382,2.0,120 w 83 st,132 w 89 st,2002,10024,912170392,2002,03M166,10024.0,MANHATTAN SCHOOL OF ARTS & TEC,ENGINE COMPANY 74 FAMILY FUND INC,Elementary
3333,19588,7391,2.0,120 w 83 st,100 w 84 st,2002,10024,912170392,2002,03M243,10024.0,THE CENTER SCHOOL,ENGINE COMPANY 74 FAMILY FUND INC,Junior High-Intermediate-Middle
64,19580,4514,2.0,1270 huguenot ave,1270 huguenot av,1999,10312,911960939,1999,31R007,10312.0,ELIAS BERNSTEIN I.S. #7,PTA NEW YORK CONGRESS,Junior High-Intermediate-Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,32,3209,2.0,6435 102 st,63-55 102 st,1998,11374,112405496,1998,28Q157,11374.0,STEPHEN HALSEY - I.S. # 157,PARENTS ASSOCIATION OF P S 175Q,Junior High-Intermediate-Middle
763,32,3212,2.0,6435 102 st,64-35 102 st,1998,11374,112405496,1998,28Q175,11374.0,LYNN GROSS DISCOVERY SCHOOL,PARENTS ASSOCIATION OF P S 175Q,Elementary
2,2,32491,2.0,255 ionia ave,255 ionia av,2011,10312,43644362,2011,31R036,10312.0,P.S. 036 J. C. DRUMGOOLE,PTA NEW YORK CONGRESS 36 PUBLIC SCHOOL PTA,Elementary
1,1,30343,2.0,255 ionia ave,255 ionia av,2010,10312,43644362,2010,31R036,10312.0,P.S. 036 J. C. DRUMGOOLE,PTA NEW YORK CONGRESS,Elementary


In [52]:
final_merge[final_merge['level_0']==32][cols]

,level_0,level_1,Score,pta_address_Lookup,school_address_Lookup,pta_year_Lookup,pta_zip_Lookup,pta_ein_Lookup,school_year_Lookup,school_dbn_Lookup,school_zip_Lookup,school_name_Lookup,pta_name_Lookup,school_cate_Lookup
762,32,3209,2.0,6435 102 st,63-55 102 st,1998,11374,112405496,1998,28Q157,11374.0,STEPHEN HALSEY - I.S. # 157,PARENTS ASSOCIATION OF P S 175Q,Junior High-Intermediate-Middle
763,32,3212,2.0,6435 102 st,64-35 102 st,1998,11374,112405496,1998,28Q175,11374.0,LYNN GROSS DISCOVERY SCHOOL,PARENTS ASSOCIATION OF P S 175Q,Elementary


In [53]:
final_merge.sort_values(by=['level_0', 'Score'],
                    ascending=False).to_excel('merge_list.xlsx',
                                              index=False)
